## Resources

This file is just a basic example to get started.  There are lots of other references.  A few:

* https://www.digitalocean.com/community/tutorials/how-to-use-web-apis-in-python-3
* https://www.dataquest.io/blog/python-api-tutorial/
* https://www.codecademy.com/courses/python-intermediate-en-6zbLp/0/1
* http://socialmedia-class.org/twittertutorial.html

And the requests package documentation: http://docs.python-requests.org/en/master/

In [1]:
import json
import requests
import pandas as pd

## Deck of Cards API

We're going to call the deck of cards API: https://deckofcardsapi.com/

In [2]:
baseurl = "https://deckofcardsapi.com/api/deck"

To get a deck, we need to call: https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=1

In [3]:
response = requests.get(baseurl+"/new/shuffle/?deck_count=1")

In [4]:
response.content

b'{"remaining": 52, "shuffled": true, "success": true, "deck_id": "dr0pogjcw6q6"}'

Or we could have constructed the url like:

In [5]:
response = requests.get(baseurl+"/new/shuffle/", params={'deck_count':1})
response.content

b'{"remaining": 52, "shuffled": true, "success": true, "deck_id": "14uzl95zowr4"}'

The above looks like it might be a dict, but it's not accessible like that:

In [6]:
response.content["deck_id"]

TypeError: byte indices must be integers or slices, not str

We need to get the JSON - the code below will automatically get us just the content

In [7]:
response.json()

{'deck_id': '14uzl95zowr4', 'remaining': 52, 'shuffled': True, 'success': True}

In [8]:
deck = response.json()

Now let's draw some cards.  We want need the `deck_id` from above to put in https://deckofcardsapi.com/api/deck/[deckid]/draw/?count=2

In [9]:
cardsresponse = requests.get("{}/{}/draw".format(baseurl, deck["deck_id"]), params={"count":2})
cardsresponse.json()

{'cards': [{'code': '0H',
   'image': 'http://deckofcardsapi.com/static/img/0H.png',
   'images': {'png': 'http://deckofcardsapi.com/static/img/0H.png',
    'svg': 'http://deckofcardsapi.com/static/img/0H.svg'},
   'suit': 'HEARTS',
   'value': '10'},
  {'code': '2H',
   'image': 'http://deckofcardsapi.com/static/img/2H.png',
   'images': {'png': 'http://deckofcardsapi.com/static/img/2H.png',
    'svg': 'http://deckofcardsapi.com/static/img/2H.svg'},
   'suit': 'HEARTS',
   'value': '2'}],
 'deck_id': '14uzl95zowr4',
 'remaining': 50,
 'success': True}

That's great, but now what do we do with that?  We could just keep the object and append more results to it for use later.  But perhaps we're trying to make a data set.  We could store the cards part of the response into a pandas data frame

In [10]:
df = pd.DataFrame(cardsresponse.json()['cards'])
df

,code,image,images,suit,value
0,0H,http://deckofcardsapi.com/static/img/0H.png,{'png': 'http://deckofcardsapi.com/static/img/...,HEARTS,10
1,2H,http://deckofcardsapi.com/static/img/2H.png,{'png': 'http://deckofcardsapi.com/static/img/...,HEARTS,2


Let's add more cards from the deck

In [11]:
def get_cards(n, deckid):
    cardsresponse = requests.get("{}/{}/draw".format(baseurl, deckid), params={"count":n})
    return pd.DataFrame(cardsresponse.json()['cards'])

for n in [5,3,2]: # get 5 cards, 3, cards, 2 cards
    df = df.append(get_cards(n, deck["deck_id"]))
    
df

,code,image,images,suit,value
0,0H,http://deckofcardsapi.com/static/img/0H.png,{'png': 'http://deckofcardsapi.com/static/img/...,HEARTS,10
1,2H,http://deckofcardsapi.com/static/img/2H.png,{'png': 'http://deckofcardsapi.com/static/img/...,HEARTS,2
0,0S,http://deckofcardsapi.com/static/img/0S.png,{'png': 'http://deckofcardsapi.com/static/img/...,SPADES,10
1,QC,http://deckofcardsapi.com/static/img/QC.png,{'png': 'http://deckofcardsapi.com/static/img/...,CLUBS,QUEEN
2,6C,http://deckofcardsapi.com/static/img/6C.png,{'png': 'http://deckofcardsapi.com/static/img/...,CLUBS,6
3,4D,http://deckofcardsapi.com/static/img/4D.png,{'png': 'http://deckofcardsapi.com/static/img/...,DIAMONDS,4
4,JC,http://deckofcardsapi.com/static/img/JC.png,{'png': 'http://deckofcardsapi.com/static/img/...,CLUBS,JACK
0,KS,http://deckofcardsapi.com/static/img/KS.png,{'png': 'http://deckofcardsapi.com/static/img/...,SPADES,KING
1,4H,http://deckofcardsapi.com/static/img/4H.png,{'png': 'http://deckofcardsapi.com/static/img/...,HEARTS,4
2,8S,http://deckofcardsapi.com/static/img/8S.png,{'png': 'http://deckofcardsapi.com/static/img/...,SPADES,8


What happens if there aren't enough cards left?

In [12]:
cardsresponse = requests.get("{}/{}/draw".format(baseurl, deck["deck_id"]), params={"count":50})

In [14]:
cardsresponse.json()

{'cards': [{'code': '4S',
   'image': 'http://deckofcardsapi.com/static/img/4S.png',
   'images': {'png': 'http://deckofcardsapi.com/static/img/4S.png',
    'svg': 'http://deckofcardsapi.com/static/img/4S.svg'},
   'suit': 'SPADES',
   'value': '4'},
  {'code': '5S',
   'image': 'http://deckofcardsapi.com/static/img/5S.png',
   'images': {'png': 'http://deckofcardsapi.com/static/img/5S.png',
    'svg': 'http://deckofcardsapi.com/static/img/5S.svg'},
   'suit': 'SPADES',
   'value': '5'},
  {'code': '9S',
   'image': 'http://deckofcardsapi.com/static/img/9S.png',
   'images': {'png': 'http://deckofcardsapi.com/static/img/9S.png',
    'svg': 'http://deckofcardsapi.com/static/img/9S.svg'},
   'suit': 'SPADES',
   'value': '9'},
  {'code': '3H',
   'image': 'http://deckofcardsapi.com/static/img/3H.png',
   'images': {'png': 'http://deckofcardsapi.com/static/img/3H.png',
    'svg': 'http://deckofcardsapi.com/static/img/3H.svg'},
   'suit': 'HEARTS',
   'value': '3'},
  {'code': 'JS',
   'im

The response type was fine -- we got data back.  It drew the remaining cards then added an error that there weren't enough left in the deck.  This could be tricky to detect.